In [1]:
from sklearn.preprocessing import *
from sklearn.feature_extraction.text import *
import numpy as np
import pandas as pd
from tensorflow import keras
import scipy.sparse as sps
import tensorflow as tf

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
from util import *

In [4]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")

In [5]:
# text_vectorizer = TfidfVectorizer(min_df=5)

# abstract_vectors = text_vectorizer.fit_transform(training_set.abstract)
# test_abstract_vectors = text_vectorizer.transform(testing_set.abstract)

# text_vectorizer = TfidfVectorizer(min_df=5)

# claim_vectors = text_vectorizer.fit_transform(training_set.claims)
# test_claim_vectors = text_vectorizer.transform(testing_set.claims)

# text_vectorizer = TfidfVectorizer(min_df=2, max_df=0.5)

# desc_vectors = text_vectorizer.fit_transform(training_set.description)
# test_desc_vectors = text_vectorizer.transform(testing_set.description)

In [95]:
subset = list(set(tier1) & set(all_tiers_100))
subset

['AnalysisAndModeling',
 'Manufacturing',
 'SpecificationofUse',
 'SurgicalMethod',
 'Imaging',
 'AnatomicalTarget']

In [96]:
labels = training_set[subset].values.astype(int)
test_labels = testing_set[subset].values.astype(int)

In [97]:
labels.shape

(972, 6)

In [98]:
from transformers import *
from tensorflow import keras

In [99]:
from tensorflow.keras import layers, models

In [74]:
from transformers import TFAutoModel, AutoTokenizer
from tqdm.auto import tqdm


# # def tokenize(sentences, tokenizer):
# #     input_ids, input_masks, input_segments = [],[],[]
# #     for sentence in tqdm(sentences):
# #         inputs = tokenizer.encode_plus(sentence,
# #                                        truncation=True,
# #                                        add_special_tokens=True, 
# #                                        max_length=256,
# #                                        padding=True,
# #                                        return_tensors='tf',
# #                                        #pad_to_max_length=True,
# #                                        return_attention_mask=False, 
# #                                        return_token_type_ids=False)
# #         input_ids.append(inputs['input_ids'])
# #         input_masks.append(inputs['attention_mask'])
# #         input_segments.append(inputs['token_type_ids'])        
        
#     return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')


In [75]:
tokenizer = AutoTokenizer.from_pretrained("/var/patentmark/patentBERT/")
transformer = TFAutoModel.from_pretrained("/var/patentmark/patentBERT/", from_pt=True)
config = AutoConfig.from_pretrained("/var/patentmark/patentBERT/")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['classifier.bias', 'classifier.weight', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [76]:
max_length=256

In [77]:
claim_input_ids = tokenizer(text=training_set.claims.to_list(),
                            truncation=True,
                           add_special_tokens=True, 
                           max_length=max_length,
                           padding=True,
                           return_tensors='tf',
                           #pad_to_max_length=True,
                           return_attention_mask=False, 
                           return_token_type_ids=False)

In [78]:
test_claim_input_ids = tokenizer(text=testing_set.claims.to_list(),
                            truncation=True,
                           add_special_tokens=True, 
                           max_length=max_length,
                           padding=True,
                           return_tensors='tf',
                           #pad_to_max_length=True,
                           return_attention_mask=False, 
                           return_token_type_ids=False)

In [79]:
import tensorflow_addons as tfa

In [80]:
# claim_input_ids, claim_masks, _ = tokenize(training_set.claims, tokenizer)
# test_claim_input_ids, test_claim_masks, _ = tokenize(testing_set.claims, tokenizer)

In [102]:
def create_model():
    #claim_input = layers.Input(shape=(claim_vectors.shape[1]))
    #description_input = layers.Input(shape=(desc_vectors.shape[1]))
    
    #claim_ids_in = layers.Input(shape=(max_length,), dtype='int32')
    #claim_masks_in = layers.Input(shape=(claim_masks.shape[1]), dtype='int32')
    #claim_embedding_layer = transformer(claim_ids_in)[1]
    #, attention_mask=claim_masks_in)[0]
    #claim_embedding_layer = layers.GlobalAveragePooling1D()(claim_embedding_layer)
    
    #concat_layer = layers.Concatenate()([claim_input])
    #dropout = layers.Dropout(config.hidden_dropout_prob)(claim_embedding_layer, training=False)
    
    abstract_input = layers.Input(shape=(abstract_vectors.shape[1]), name="abstract_vectors")
    claim_input = layers.Input(shape=(claim_vectors.shape[1]), name="claim_vectors")
    
    concat = layers.Concatenate()((abstract_input, claim_input))
    dropout0 = layers.Dropout(0.7)(concat)
    dense0 = layers.Dense(64, activation='elu')(dropout0)
    dropout1 = layers.Dropout(0.7)(dense0)
    output = layers.Dense(units=len(subset), activation='sigmoid')(dropout1)
    
                          #kernel_initializer=keras.initializers.TruncatedNormal(stddev=config.initializer_range))(dense0)
    
    #, activation='sigmoid')(dropout)
    
    #optimizer = tfa.optimizers.AdamW(weight_decay=0.01, learning_rate=1e-05, epsilon=1e-08)
    loss = keras.losses.BinaryCrossentropy()
    #metric = keras.metrics.CategoricalAccuracy('accuracy')
    metric = tfa.metrics.HammingLoss(mode='multilabel', threshold=0.5)
    #metric = 'categorical_accuracy'
    
    net = models.Model([claim_input, abstract_input], output)
    
    net.compile(loss=loss, metrics=[metric], optimizer="adam")
#     for layer in net.layers[:3]:
#         layer.trainable = False
    return net

In [103]:
model = create_model()

In [104]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
abstract_vectors (InputLayer)   [(None, 1275)]       0                                            
__________________________________________________________________________________________________
claim_vectors (InputLayer)      [(None, 2845)]       0                                            
__________________________________________________________________________________________________
concatenate_7 (Concatenate)     (None, 4120)         0           abstract_vectors[0][0]           
                                                                 claim_vectors[0][0]              
__________________________________________________________________________________________________
dropout_88 (Dropout)            (None, 4120)         0           concatenate_7[0][0]        

In [105]:
from datetime import datetime
from tensorflow.keras.callbacks import * 

logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

callbacks = [
        ReduceLROnPlateau(monitor='val_hamming_loss'),
        EarlyStopping(patience=30, monitor='val_hamming_loss'),
        ModelCheckpoint(filepath="keras-model", save_best_only=True),
        tensorboard_callback
    ]

model.fit(x={"claim_vectors": claim_vectors.todense(),
             "abstract_vectors": abstract_vectors.todense()}, 
          y=labels,
          verbose=2, 
          epochs=100, 
          validation_split=0.2, 
          batch_size=1024,
          callbacks=callbacks)

Epoch 1/100
INFO:tensorflow:Assets written to: keras-model/assets
1/1 - 0s - loss: 0.6978 - hamming_loss: 0.5159 - val_loss: 0.6896 - val_hamming_loss: 0.4487 - lr: 0.0010
Epoch 2/100
INFO:tensorflow:Assets written to: keras-model/assets
1/1 - 0s - loss: 0.6906 - hamming_loss: 0.4683 - val_loss: 0.6833 - val_hamming_loss: 0.3940 - lr: 0.0010
Epoch 3/100
INFO:tensorflow:Assets written to: keras-model/assets
1/1 - 0s - loss: 0.6842 - hamming_loss: 0.4397 - val_loss: 0.6774 - val_hamming_loss: 0.3538 - lr: 0.0010
Epoch 4/100
INFO:tensorflow:Assets written to: keras-model/assets
1/1 - 0s - loss: 0.6794 - hamming_loss: 0.4215 - val_loss: 0.6718 - val_hamming_loss: 0.3521 - lr: 0.0010
Epoch 5/100
INFO:tensorflow:Assets written to: keras-model/assets
1/1 - 0s - loss: 0.6745 - hamming_loss: 0.4005 - val_loss: 0.6666 - val_hamming_loss: 0.3487 - lr: 0.0010
Epoch 6/100
INFO:tensorflow:Assets written to: keras-model/assets
1/1 - 0s - loss: 0.6695 - hamming_loss: 0.3837 - val_loss: 0.6617 - val_ha

1/1 - 0s - loss: 0.6034 - hamming_loss: 0.3177 - val_loss: 0.6091 - val_hamming_loss: 0.3256 - lr: 1.0000e-05
Epoch 49/100
INFO:tensorflow:Assets written to: keras-model/assets
1/1 - 0s - loss: 0.5994 - hamming_loss: 0.3181 - val_loss: 0.6090 - val_hamming_loss: 0.3256 - lr: 1.0000e-05
Epoch 50/100
INFO:tensorflow:Assets written to: keras-model/assets
1/1 - 0s - loss: 0.6010 - hamming_loss: 0.3269 - val_loss: 0.6090 - val_hamming_loss: 0.3256 - lr: 1.0000e-05
Epoch 51/100
INFO:tensorflow:Assets written to: keras-model/assets
1/1 - 0s - loss: 0.6006 - hamming_loss: 0.3235 - val_loss: 0.6090 - val_hamming_loss: 0.3256 - lr: 1.0000e-05
Epoch 52/100
INFO:tensorflow:Assets written to: keras-model/assets
1/1 - 0s - loss: 0.6020 - hamming_loss: 0.3211 - val_loss: 0.6090 - val_hamming_loss: 0.3256 - lr: 1.0000e-05
Epoch 53/100
INFO:tensorflow:Assets written to: keras-model/assets
1/1 - 0s - loss: 0.6031 - hamming_loss: 0.3192 - val_loss: 0.6090 - val_hamming_loss: 0.3256 - lr: 1.0000e-05
Epoch

In [106]:
labels[0]

array([1, 1, 1, 0, 1, 1])

In [107]:
preds = model.predict({"abstract_vectors": abstract_vectors.todense(),
                       "claim_vectors": claim_vectors.todense()})

In [108]:
preds[preds>=0.5] = 1
preds[preds<0.5] = 0

In [109]:
from sklearn.metrics import classification_report

In [110]:
print(classification_report(labels, preds, target_names=subset))

                     precision    recall  f1-score   support

AnalysisAndModeling       0.00      0.00      0.00       342
      Manufacturing       1.00      0.04      0.08       387
 SpecificationofUse       0.00      0.00      0.00       347
     SurgicalMethod       0.00      0.00      0.00       161
            Imaging       0.64      0.95      0.77       517
   AnatomicalTarget       0.63      1.00      0.77       606

          micro avg       0.64      0.47      0.54      2360
          macro avg       0.38      0.33      0.27      2360
       weighted avg       0.47      0.47      0.38      2360
        samples avg       0.66      0.49      0.53      2360



In [111]:
test_preds =model.predict({"claim_vectors": test_claim_vectors.todense(),
                           "abstract_vectors": test_abstract_vectors.todense()})

In [112]:
test_preds[test_preds>=0.5] = 1
test_preds[test_preds<0.5] = 0

In [113]:
print(classification_report(test_labels, test_preds, target_names=subset))

                     precision    recall  f1-score   support

AnalysisAndModeling       0.00      0.00      0.00        84
      Manufacturing       0.00      0.00      0.00        83
 SpecificationofUse       0.00      0.00      0.00        79
     SurgicalMethod       0.00      0.00      0.00        40
            Imaging       0.59      0.92      0.72       133
   AnatomicalTarget       0.68      0.99      0.80       164

          micro avg       0.63      0.49      0.55       583
          macro avg       0.21      0.32      0.25       583
       weighted avg       0.32      0.49      0.39       583
        samples avg       0.65      0.52      0.55       583

